In [ ]:
!pip install langchain wikipedia langchain-community
!pip install langchain-groq langchain-cohere
!pip install pydantic duckduckgo-search

In [ ]:
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor

from datetime import datetime
from pydantic import BaseModel

In [ ]:
search = DuckDuckGoSearchRun()
search_tool = Tool(name="search", func=search.run, description="Search the web for information, especially the current weather", tool_type="search")

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_characters_max=50)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
def print_time(data: str):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Time now is: {timestamp}\n\n"
################### WRAP THE ABOVE PYTHON FUNCTION AS A TOOL FOR LANGCHAIN, AS BELOW
time_tool = Tool(
    name="print_current_time",
    func=print_time,
    description="Prints the current time.",
)

In [ ]:
GROQ_API_KEY = 'GROQ_API_KEY'
COHERE_API_KEY = 'COHERE_API_KEY'

In [ ]:
llm = ChatCohere(model='command-a-03-2025',cohere_api_key=COHERE_API_KEY)
llm2 = ChatGroq(model='deepseek-r1-distill-qwen-32b',groq_api_key=GROQ_API_KEY)

In [ ]:
quick_response = llm.invoke("Tell me a joke")
print(quick_response)

content="Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄" additional_kwargs={'id': '598a5e60-f37f-4538-9fa6-5c9d72c1ec9f', 'finish_reason': 'COMPLETE', 'content': "Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄", 'token_count': {'input_tokens': 499.0, 'output_tokens': 47.0}} response_metadata={'id': '598a5e60-f37f-4538-9fa6-5c9d72c1ec9f', 'finish_reason': 'COMPLETE', 'content': "Sure! Here's a joke for you:\n\n**Why don’t skeletons fight each other?**\n\nBecause they don’t have the guts! \n\nHope that brought a smile to your face! 😄", 'token_count': {'input_tokens': 499.0, 'output_tokens': 47.0}} id='run-f4c8a4a8-84f5-4ba4-a8a7-f57ee245abf4-0' usage_metadata={'input_tokens': 499, 'output_tokens': 47, 'total_tokens': 546}


In [ ]:
class ResearchResponse(BaseModel):
    summary: str
    topic: str
    tools_used: list[str]
    sources: list[str]

parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You  are a helpful assistant. You must do 3 things in the following order.
            (1) TELL A FROG JOKE,  (2) PRINT THE CURRENT IME by calling the "print_current_time" tool,
            then (3) ANSWER USER QUERY
            Wrap output in this format.\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())
tools = [search_tool, wiki_tool, time_tool]
agent = create_tool_calling_agent(
    llm=llm2,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(agent=agent, tools=tools)
query = input("What can I help you with? ")
raw_response = agent_executor.invoke({"query": query})
#print(raw_response)


try:
    response = parser.parse(raw_response.get('output',''))
    print(response)
    print("Topic only is:", response.topic)
except Exception as e:
    print("Error", e, "Raw response", raw_response)


What can I help you with? London
summary='Why did the frog never donate blood? Because he was cold-blooded! The current time is 3:45 PM. London is a major city in the UK, known for landmarks like the Tower of London and Buckingham Palace.' topic='London' tools_used=['print_current_time', 'wikipedia'] sources=['https://en.wikipedia.org/wiki/London']
Topic only is: London
